In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('ggplot')
import numpy as np
np.random.seed(555)
import pystan

In [2]:
data_mu = [3e3, 12e3]
data_std = [5e2, 3e3]
mu_prior = 35e3
std_prior = 75e2

# true_price = np.random.normal(mu_prior, std_prior)

In [7]:
stan_code = """

parameters {
    real<lower=0> true_price;
    real<lower=0> prize_1;
    real<lower=0> prize_2;
}

transformed parameters {
    real<lower=0> price_estimate;
    price_estimate = prize_1 + prize_2;
}

model {
    true_price ~ normal(35e3, 75e2);
    prize_1 ~ normal(3e3, 5e2);
    prize_2 ~ normal(12e3, 12e3);

    target += normal_lpdf(true_price | 35e3, 75e2);
}
"""

In [8]:
model = pystan.StanModel(model_code=stan_code)

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_7b70111ff47034aa2a2d4d1730c695ec NOW.


In [9]:
fit = model.sampling(iter=1000, chains=4, seed=555, thin=1, warmup=100, n_jobs=-1)
fit

Inference for Stan model: anon_model_7b70111ff47034aa2a2d4d1730c695ec.
4 chains, each with iter=1000; warmup=100; thin=1; 
post-warmup draws per chain=900, total post-warmup draws=3600.

                 mean se_mean     sd   2.5%    25%    50%    75%  97.5%  n_eff   Rhat
true_price      3.5e4   87.02 5221.1  2.5e4  3.2e4  3.5e4  3.9e4  4.5e4   3600    1.0
prize_1        3002.6    8.97 500.37 1999.1 2676.1 3004.5 3340.0 3947.6   3115    1.0
prize_2         1.6e4  429.07 9756.0 984.28 7886.6  1.4e4  2.2e4  3.6e4    517   1.01
price_estimate  1.9e4  430.02 9777.6 3921.6  1.1e4  1.7e4  2.5e4  3.9e4    517   1.01
lp__             16.6    0.06   1.37  13.27  15.88  16.97  17.64  18.21    584    1.0

Samples were drawn using NUTS at Wed Feb  7 18:00:43 2018.
For each parameter, n_eff is a crude measure of effective sample size,
and Rhat is the potential scale reduction factor on split chains (at 
convergence, Rhat=1).